In [ ]:
"""
Author: Tharnier O. Puel
Last Modified: Dec 2024

Description: 
    Utilizes Optuna to perform a parameterized fitting of the Hamiltonian for the trivalent lanthanide ions.

License: 
    All rights reserved to proprietary.

Contact: 
    Email  : tharnier@me.com
    Website: https://topuel.wordpress.com
    GitHub : https://github.com/Tharnier
    

Editable Sections:
    - Cell [# Import Hamiltonian matrices from files]:
        Adjust the directory path:
        [directory = 'symbolic_Hamiltonian_Er/']
    
    - Cell [# Prepare target eigenvalues]:
        Adjust the file path:
        [with open("Experimental_energies/Er-spectrum.m", "r") as file:]
    
    - Cell [# Function to assemble the parametrized Hamiltonian]:
        Modify trial parameter declaration lines:
        [param = trial.suggest_float('Param', 1234., 2345.)]
        Update fitting_params = {dictionary} to include all declared trial parameters.
    
    - Cell [# Run optuna]:
        Adjust lines for:
        [n_trials = 123]
        [fixed_params = {dictionary}]
        [ratios = {dictionary}]
    
    - Cell [# Save results to files]:
        Modify directory and file name for results storage:
        [directory = 'fitting-trials/']
        [file_name = 'Er_Optuna']

Notes: 
    - Ensure all paths are valid before running the notebook.
    - Review editable sections before executing cells to avoid runtime errors.
"""

In [1]:
# ----------------
# Import libraries 
# ----------------

import os      as os # Handle file's paths and names
import torch   as pt # Tensor algebra, used for matrix diagonalization
import pandas  as pd # Handle mixed data types and used to save .CSV files
import optuna  as op # Parameters optimization

In [2]:
# --------------------------------------
# Import Hamiltonian matrices from files
# --------------------------------------

# Directory containing symbolic Hamiltonian files
directory = 'symbolic_Hamiltonian_Er/'

# Store file contents as expressions
file_expressions = {}

# Process files (ended with .m) and store content in the dictionary
for file_name in os.listdir(directory):
    if file_name.endswith('.m'):
        file_path = os.path.join(directory, file_name)
        with open(file_path, 'r') as file:
            # Read file and strip unnecessary spaces/newlines
            content = file.read().strip().replace("\n","")
            # Remove headers created by Mathematica
            content = content.replace("(* Created with the Wolfram Language : www.wolfram.com *)", "").replace("(* ::Package:: *)", "")
            # Create dictionary key
            key = file_name.replace("Er_matrix_for_python_","").replace(".m","")
            # Update dictionary
            file_expressions[key] = content

# Output the dictionary keys
print( file_expressions.keys() )

dict_keys(['B46', 'S16', 'B02', 'S56', 'B06', 'S12', 'S46', 'B16', 'T16', 'F0', 'T3', 'T18', 'F4', 'T7', 'B56', 'T12', 'B12', 'Zeta', 'M4', 'B24', 'Gamma', 'S34', 'M0', 'T15', 'Beta', 'P4', 'S24', 'B34', 'T4', 'T11', 'S14', 'B44', 'B04', 'T11p', 'F2', 'T14', 'B14', 'S44', 'Alpha', 'F6', 'T2p', 'B26', 'M2', 'B66', 'S36', 'B22', 'P6', 'S26', 'T2', 'T8', 'T17', 'T6', 'S66', 'T19', 'B36', 'S22', 'P2'])


In [3]:
# -----------------------------------------------------
# Function to convert Mathematica expressions to Python
# ----------------------------------------------------- 

def convert_mathematica_to_Python(matrix_string, type='pytorch'):
    """
    Converts a Mathematica-style matrix string to a PyTorch tensor or Pandas data.

    Args:
        matrix_string (str): The Mathematica-style matrix string.

    Returns:
        pt.tensor or pd.DataFrame: The converted matrix in either PyTorch of Pandas format.
    """
    # Replace Mathematica-specific syntax with Python-compatible syntax
    python_style = (
        matrix_string
        .replace('Sqrt[', 'pt.tensor(')  # Convert Mathematica Sqrt[...] to PyTorch pt.tensor(...).sqrt()
        .replace(']',').sqrt()')         # Convert Mathematica Sqrt[...] to PyTorch pt.tensor(...).sqrt()
        .replace('{', '[')               # Convert Mathematica braces to Python brackets
        .replace('}', ']')               # Convert Mathematica braces to Python brackets
        .replace('I', '1j')              # Convert Mathematica complex I to Python 1j
        .replace('*^', 'e')              # Convert Mathematica scientific notation
    )
    
    # Parse and execute expression provided as a string
    matrix_list = eval(python_style, {"pt" : pt, "j": 1j})

    # Convert to Pandas data or PyTorch tensor
    if type == 'pandas':
        output = pd.DataFrame(matrix_list)
    elif type == 'pytorch':
        output = pt.tensor(matrix_list, dtype=pt.complex128)
    else:
        print('From convert_mathematica_to_Python(): invalid option (type)')

    # Convert to PyTorch tensor
    return output

In [4]:
# -------------------------
# Prepare target eigenvalues
# -------------------------

# Open and read eigenvalues file
with open("Experimental_energies/Er-spectrum.m", "r") as file:
    # Read file and strip unnecessary spaces/newlines
    content = file.read().strip().replace("\n","")
    # Remove headers created by Mathematica
    file_expressions_target_eigenvalues = content.replace("(* Created with the Wolfram Language : www.wolfram.com *)", "").replace("(* ::Package:: *)", "")

# Define the Mathematica-like expression as a string
target_eigenvalues = convert_mathematica_to_Python( file_expressions_target_eigenvalues, type='pandas' )

# Find numerical values
mask = target_eigenvalues.applymap(lambda value: isinstance(value, float))
# Filter target eigenvalues to numeric values only
numeric_target_eigenvalues = target_eigenvalues[mask].dropna().astype(float)

# Convert indices and values to PyTorch tensors
target_index_set          = pt.tensor(numeric_target_eigenvalues.index , dtype=pt.long).flatten()
target_eigenvalues_tensor = pt.tensor(numeric_target_eigenvalues.values, dtype=pt.float64).flatten()

# Shift and sort energies to start at zero
target_eigenvalues_tensor, idx = pt.sort( target_eigenvalues_tensor - pt.min(target_eigenvalues_tensor) )

# Display imported values
print( target_eigenvalues_tensor )
print( target_index_set )

tensor([    0.0000,     0.0000,    51.2000,    51.2000,   121.2000,   121.2000,
          199.7000,   199.7000,   219.4000,   219.4000,   313.8000,   313.8000,
          400.3000,   400.3000,   442.9000,   442.9000,  6604.0000,  6604.0000,
         6630.0000,  6630.0000,  6670.0000,  6670.0000,  6700.0000,  6700.0000,
         6723.0000,  6723.0000,  6754.0000,  6754.0000,  6823.0000,  6823.0000,
        10301.0000, 10301.0000, 10311.0000, 10311.0000, 10330.0000, 10330.0000,
        10344.0000, 10344.0000, 10358.0000, 10358.0000, 10395.0000, 10395.0000,
        12419.0000, 12419.0000, 12518.0000, 12518.0000, 12615.0000, 12615.0000,
        12701.0000, 12701.0000, 12730.0000, 12730.0000, 15391.0000, 15391.0000,
        15432.0000, 15432.0000, 15443.0000, 15443.0000, 15474.0000, 15474.0000,
        15527.0000, 15527.0000, 18557.0000, 18557.0000, 18588.0000, 18588.0000,
        19266.0000, 19266.0000, 19307.0000, 19307.0000, 19314.0000, 19314.0000,
        19363.0000, 19363.0000, 19367.00

In [5]:
# -------------------------------------------
# Convert Mathematica expressions into Python
# -------------------------------------------

# Apply Mathematica to Python convertion to the files expressions
matrices_dictionary = {key: convert_mathematica_to_Python(value, type='pytorch') for key, value in file_expressions.items()}

# Output the dictionary keys
print( matrices_dictionary.keys() )

dict_keys(['B46', 'S16', 'B02', 'S56', 'B06', 'S12', 'S46', 'B16', 'T16', 'F0', 'T3', 'T18', 'F4', 'T7', 'B56', 'T12', 'B12', 'Zeta', 'M4', 'B24', 'Gamma', 'S34', 'M0', 'T15', 'Beta', 'P4', 'S24', 'B34', 'T4', 'T11', 'S14', 'B44', 'B04', 'T11p', 'F2', 'T14', 'B14', 'S44', 'Alpha', 'F6', 'T2p', 'B26', 'M2', 'B66', 'S36', 'B22', 'P6', 'S26', 'T2', 'T8', 'T17', 'T6', 'S66', 'T19', 'B36', 'S22', 'P2'])


In [6]:
# -------------------------------------------------
# Function to assemble the parametrized Hamiltonian
# -------------------------------------------------

def matrix_Hamiltonian(fitting_params, fixed_params = {}, ratios = {}):
    """
    Add matrices weighted by their coefficients (parameters).
    Notice that it will only sum over the provided parameters.

    Args:
        fitting_dictionary: Dictionary parameters to be fitted.
        fixed_params: Dictionary with parameters left out from the fitting.
        ratios: { 'P4/P2': value, 'P6/P2': value, 'M2/M0': value, 'M4/M0': value }

    Returns:
        pt.tensor: The assembled matrix as PyTorch tensor.
    """
    # CHECK if the same param appears in both dictionaries
    common_keys = fitting_params.keys() & fixed_params.keys() | ratios.keys() & fitting_params.keys() | ratios.keys() & fixed_params.keys()
    if common_keys:
        raise SystemExit(f'From matrix_Hamiltonian(). The following params appear in multiple dictionaries: {common_keys}')

    # CHECK if keys provided in fitting_params and fixed_params exist in Hamiltonian (matrices_dictionary)
    if not common_keys.issubset(matrices_dictionary.keys()):
        missing_keys = common_keys - matrices_dictionary.keys()
        raise SystemExit(f'From matrix_Hamiltonian(). The following parameter keys are missing in the Hamiltonian: {missing_keys}')

    # Multiply matrices by their corresponding parameters and return their sum (only over the provided parameters)
    numerical_matrix = sum(fitting_params[key] * matrices_dictionary[key] for key in fitting_params)
    if fixed_params:
        numerical_matrix += sum(fixed_params[key] * matrices_dictionary[key] for key in fixed_params)

    # Add P4 and P6 parameters as ratios to P2
    if {'P4/P2'}.issubset(ratios.keys()) or {'P6/P2'}.issubset(ratios.keys()):
        if   not {'P4/P2','P6/P2'}.issubset(ratios.keys()):
            raise SystemExit('From matrix_Hamiltonian(). Both keys \'P4/P2\' and \'P6/P2\' must be defined')
        elif not {'P2'}.issubset(fitting_params.keys()):
            raise SystemExit('From matrix_Hamiltonian(). The ratios option only accepts P2 as a fitting parameter')
        elif {'P4'} & fitting_params.keys() | {'P4'} & fixed_params.keys() | {'P6'} & fitting_params.keys() | {'P6'} & fixed_params.keys():
            raise SystemExit('From matrix_Hamiltonian(). Parameters P4 and P6 cannot be defined if using P4/P2 and P6/P2 ratios')
        elif not {'P4','P6'}.issubset(matrices_dictionary.keys()):
            raise SystemExit('From matrix_Hamiltonian(). One or more of the following parameter keys are missing in the Hamiltonian: P4 and P6')
        else:
            numerical_matrix += fitting_params['P2']*ratios['P4/P2']*matrices_dictionary['P4'] + fitting_params['P2']*ratios['P6/P2']*matrices_dictionary['P6']
    
    # Add M2 and M4 parameters as ratios to M0
    if {'M2/M0'}.issubset(ratios.keys()) or {'M4/M0'}.issubset(ratios.keys()):
        if   not {'M2/M0','M4/M0'}.issubset(ratios.keys()):
            raise SystemExit('From matrix_Hamiltonian(). Both keys \'M2/M0\' and \'M4/M0\' must be defined')
        elif   not {'M0'}.issubset(fitting_params.keys()):
            raise SystemExit('From matrix_Hamiltonian(). The ratios option only accepts M0 as a fitting parameter')
        elif {'M2'} & fitting_params.keys() | {'M2'} & fixed_params.keys() | {'M4'} & fitting_params.keys() | {'M4'} & fixed_params.keys():
            raise SystemExit('From matrix_Hamiltonian(). Parameters M2 and M4 cannot be defined if using M2/M0 and M4/M0 ratios')
        elif not {'M2','M4'}.issubset(matrices_dictionary.keys()):
            raise SystemExit('From matrix_Hamiltonian(). One or more of the following parameter keys are missing in the Hamiltonian: M2 and M4')
        else:
            numerical_matrix += fitting_params['M0']*ratios['M2/M0']*matrices_dictionary['M2'] + fitting_params['M0']*ratios['M4/M0']*matrices_dictionary['M4']

    return numerical_matrix

In [7]:
# -----------------------------
# Objective function for optuna
# -----------------------------

def actual_objective(trial, fixed_params = {}, ratios = {}):
    # Suggested values range for parameters (see Carnall et al. J.Chem.Phys. 90, 3443 (1989))
    f2    = trial.suggest_float('F2'   , 97400., 97500.) # ErLaF3: 97483.
    f4    = trial.suggest_float('F4'   , 67850., 67950.) # ErLaF3: 67904.
    f6    = trial.suggest_float('F6'   , 53950., 54050.) # ErLaF3: 54010.
    zeta  = trial.suggest_float('Zeta' , 2370. , 2380. ) # ErLaF3: 2376.
    alpha = trial.suggest_float('Alpha', 15.   , 20.   ) # ErLaF3: 17.79
    beta  = trial.suggest_float('Beta' , -600. , -500. ) # ErLaF3: -582.1
    t3    = trial.suggest_float('T3'   , 40.   , 50.   ) # ErLaF3: 43.
    t4    = trial.suggest_float('T4'   , 70.   , 80.   ) # ErLaF3: 73.
    t6    = trial.suggest_float('T6'   , -300. , -200. ) # ErLaF3: -271.
    t7    = trial.suggest_float('T7'   , 250.  , 350.  ) # ErLaF3: 308.
    t8    = trial.suggest_float('T8'   , 250.  , 350.  ) # ErLaF3: 299.
    m0    = trial.suggest_float('M0'   , 3.    , 4.    ) # ErLaF3: 3.86
    p2    = trial.suggest_float('P2'   , 550.  , 650.  ) # ErLaF3: 594.
    b02   = trial.suggest_float('B02'  , -300. , -200. ) # ErLaF3: -238.
    b04   = trial.suggest_float('B04'  , 400.  , 500.  ) # ErLaF3: 453.
    b06   = trial.suggest_float('B06'  , 300.  , 400.  ) # ErLaF3: 373.
    b22   = trial.suggest_float('B22'  , -100. , -50.  ) # ErLaF3: -91.
    b24   = trial.suggest_float('B24'  , 250.  , 350.  ) # ErLaF3: 308.
    b44   = trial.suggest_float('B44'  , 400.  , 500.  ) # ErLaF3: 417.
    b26   = trial.suggest_float('B26'  , -550. , -450. ) # ErLaF3: -489.
    b46   = trial.suggest_float('B46'  , -300. , -200. ) # ErLaF3: -240.
    b66   = trial.suggest_float('B66'  , -600., -500.  ) # ErLaF3: -536.

    # Parameters dictionary
    fitting_params = { 'F2': f2, 'F4': f4, 'F6': f6, 'Zeta': zeta, 'Alpha': alpha, 'Beta': beta,  \
                    'T3': t3, 'T4': t4, 'T6': t6, 'T7': t7, 'T8': t8, 'P2': p2, 'M0': m0, \
                    'B02': b02, 'B04': b04, 'B06': b06, 'B22': b22, 'B24': b24, 'B44': b44, \
                    'B26': b26, 'B46': b46, 'B66': b66 }
    
    # Assemble Hamiltonian matrix
    A_tensor = matrix_Hamiltonian(fitting_params, fixed_params, ratios)

    # Next line is optional, if one needs to use CUDA
    #A_tensor = A_tensor.to('cuda')
    
    # Use PyTorch for computing the eigenvalues
    eigenvalues = pt.linalg.eigvals(A_tensor).real.cpu()  # .cpu() is only really necessary if using CUDA
    
    # Sort and shift
    eigenvalues, idx = pt.sort( eigenvalues - pt.min(eigenvalues) )

    # Filter eigenvalues according to target values
    eigenvalues_filtered = eigenvalues[target_index_set]

    # Compute the absolute difference
    abs_diff = pt.abs(eigenvalues_filtered - target_eigenvalues_tensor)

    # Compute the denominator
    denom    = len(target_index_set) - len(fitting_params)

    # Compute the objective function, as in Carnall
    objective_value = pt.sqrt( pt.sum( abs_diff**2 ) / denom )

    # Return objective value
    return objective_value

In [8]:
# ----------
# Run optuna
# ----------

# Number of iterations
n_trials = 100

# Fixed parameters
fixed_params   = { 'Gamma': 1800., 'T2': 400. }
ratios         = { 'M2/M0': 0.56, 'M4/M0': 0.31, 'P4/P2': 0.5, 'P6/P2': 0.1 }

# Define a wrapper function to pass additional arguments to the objective function
def objective(trial):
    return actual_objective(trial, fixed_params, ratios)

# Create an Optuna study
study = op.create_study(direction='minimize')

# Optimize the study
study.optimize(objective, n_trials)

# Get the best parameters
best_fitting_params = study.best_params

[I 2024-12-19 16:07:48,875] A new study created in memory with name: no-name-5996fcae-4807-4035-8d98-7c717038ed01
[I 2024-12-19 16:07:49,051] Trial 0 finished with value: 54.579635303704485 and parameters: {'F2': 97410.29666082715, 'F4': 67883.84759640358, 'F6': 54021.18861549942, 'Zeta': 2373.9693656004247, 'Alpha': 17.900305352124363, 'Beta': -506.99232304430166, 'T3': 47.18601336745341, 'T4': 79.46747205636133, 'T6': -237.7022802886408, 'T7': 347.4906083091906, 'T8': 324.35151176681865, 'M0': 3.5313139673388054, 'P2': 574.8066653583684, 'B02': -245.65352362597935, 'B04': 426.9487004976533, 'B06': 308.5243825115806, 'B22': -89.65226868570171, 'B24': 335.11294166483134, 'B44': 490.6301191302413, 'B26': -450.2772361974319, 'B46': -230.7979892249459, 'B66': -513.1963399718957}. Best is trial 0 with value: 54.579635303704485.
[I 2024-12-19 16:07:49,178] Trial 1 finished with value: 50.164565610728715 and parameters: {'F2': 97432.67701489426, 'F4': 67916.04770480597, 'F6': 54019.789354283

In [9]:
# ---------------------
# Save results to files
# ---------------------

# Path and file
directory = 'fitting-trials/'
file_name = 'Er_Optuna'

# Export params to a plain text file
with open(directory + file_name + "_optimized_params.txt", "w") as f:
    f.write( str(best_fitting_params) )

# Recompute the eigenvalues of the optimized matrix
A_tensor = matrix_Hamiltonian(best_fitting_params, fixed_params, ratios)
#A_tensor = A_tensor.to('cuda')
optimized_eigenvalues      = pt.linalg.eigvals(A_tensor).real.cpu()
optimized_eigenvalues, idx = pt.sort( optimized_eigenvalues - pt.min(optimized_eigenvalues) )

# Export optimized eigenvalues to be read in Mathematica
optimized_eigenvalues_panda = pd.DataFrame(optimized_eigenvalues)
optimized_eigenvalues_panda.to_csv(directory + file_name + "_optimized_eigenvalues.csv", index=False, header=None)

# Convert target eigenvalues to a pandas DataFrame and Export to be read in Mathematica
target_eigenvalues_panda = pd.DataFrame(target_eigenvalues)
target_eigenvalues_panda.to_csv(directory + file_name + "_target_eigenvalues.csv", index=False, header=None)

# Export trials
best_trials_per_iteration = []
with open(directory + file_name + "_optimization_trials.txt", "w") as f:
    for trial in study.trials:
        # Format list of params output to show up to 10 decimals
        formatted_params = {key: f"{value:.10f}" for key, value in trial.params.items()}
        # Keep track of the best trial
        if trial.state == op.trial.TrialState.COMPLETE and (not best_trials_per_iteration or trial.value < best_trials_per_iteration[-1].value): 
            best_trials_per_iteration.append(trial)
        # Save trial informations to a file
        f.write( f"Time: {trial.datetime_complete} \t \
                Trial: {trial.number} \t \
                Objective value: {trial.value:.10f} \t \
                Parameters: {formatted_params} \t \
                Best trial: {best_trials_per_iteration[-1].number} \t \
                Best objective value: {best_trials_per_iteration[-1].value:.10f} \n" )

# Export optimization settings and parameters range 
with open(directory + file_name + "_search_space.txt", "w") as f:
    search_space = {key: value for trial in study.trials for key, value in trial.distributions.items()}
    f.write("Search Space: \n")
    for param_name, distribution in search_space.items():
        f.write( f"Parameter: {param_name} \t Range: {distribution} \n")
    f.write(f"\n Degrees of freedom: \n \
            Number of experimental levels: {len(target_index_set)} \n \
            Number of fitting parameters: {len(best_fitting_params)} ")
